In [23]:
import os, sys
from pathlib import Path
print("CWD:", os.getcwd())
print("sys.path[0]:", sys.path[0])


CWD: c:\Users\amanbatra\Chicken-Disease-Classification
sys.path[0]: C:\Users\amanbatra\AppData\Local\Programs\Python\Python313\python313.zip


In [2]:
%pwd

'c:\\Users\\amanbatra\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\amanbatra\\Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from Chicken_Disease_Classifier.constants import *
from Chicken_Disease_Classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:

import time

In [18]:
# ---- PrepareCallback (Path-safe for Keras 3) ----
from pathlib import Path
from datetime import datetime
import tensorflow as tf

class PrepareCallback:
    """
    Works with config fields as str OR pathlib.Path:
      - root_dir
      - tensorboard_root_log_dir
      - checkpoint_model_filepath
    Optional:
      - monitor (default "val_loss")
      - mode (default "min")
      - save_best_only (default True)
      - save_weights_only (default False)
    """

    def __init__(self, config):
        self.config = config
        self.monitor = getattr(config, "monitor", "val_loss")
        self.mode = getattr(config, "mode", "min")
        self.save_best_only = getattr(config, "save_best_only", True)
        self.save_weights_only = getattr(config, "save_weights_only", False)

        # Normalize to Path objects (handles both str and WindowsPath)
        self.cb_root = Path(getattr(config, "root_dir"))
        self.tb_root = Path(getattr(config, "tensorboard_root_log_dir"))
        self.ckpt_path = Path(getattr(config, "checkpoint_model_filepath"))

        # Ensure base dirs exist
        self.cb_root.mkdir(parents=True, exist_ok=True)
        self.tb_root.mkdir(parents=True, exist_ok=True)

        # If user passed a directory (no filename), we’ll append one
        if self.ckpt_path.suffix == "":
            # directory-like path; add a filename with correct suffix
            if self.save_weights_only:
                self.ckpt_path = self.ckpt_path / f"model-{{epoch:02d}}-{{{self.monitor}:.4f}}.weights.h5"
            else:
                self.ckpt_path = self.ckpt_path / f"model-{{epoch:02d}}-{{{self.monitor}:.4f}}.keras"
        else:
            # Has a filename; enforce correct suffix per Keras 3 rules
            if self.save_weights_only:
                if self.ckpt_path.suffix != ".h5" or not self.ckpt_path.name.endswith(".weights.h5"):
                    # replace suffix with .weights.h5 (preserve stem)
                    self.ckpt_path = self.ckpt_path.with_name(self.ckpt_path.stem + ".weights.h5")
            else:
                if self.ckpt_path.suffix != ".keras":
                    self.ckpt_path = self.ckpt_path.with_suffix(".keras")

        # Make sure checkpoint parent directory exists
        self.ckpt_path.parent.mkdir(parents=True, exist_ok=True)

        # Timestamped TB subdir
        self.tb_log_dir = self.tb_root / datetime.now().strftime("%Y%m%d-%H%M%S")
        self.tb_log_dir.mkdir(parents=True, exist_ok=True)

    def _create_tb_callback(self):
        return tf.keras.callbacks.TensorBoard(
            log_dir=str(self.tb_log_dir),  # TensorBoard expects str
            histogram_freq=1,
            write_graph=True
        )

    def _create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.ckpt_path),  # Keras expects str
            monitor=self.monitor,
            mode=self.mode,
            save_best_only=self.save_best_only,
            save_weights_only=self.save_weights_only,
            verbose=1
        )

    def get_tb_ckpt_callbacks(self):
        # Return INSTANCES, not methods
        return [
            self._create_tb_callback(),
            self._create_ckpt_callback()
        ]


In [19]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [20]:

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [26]:
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "scipy"])
import scipy; print("scipy", scipy.__version__, "installed")


scipy 1.16.2 installed


In [27]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-10-23 17:47:30,736: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-23 17:47:30,739: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-23 17:47:30,740: INFO: common: created directory at: artifacts]
[2025-10-23 17:47:30,741: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoints]
[2025-10-23 17:47:30,742: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_logs]
[2025-10-23 17:47:30,744: INFO: common: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/2
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4794 - loss: 1.3281

c:\Users\amanbatra\Chicken-Disease-Classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_loss improved from None to 1.39454, saving model to artifacts\prepare_callbacks\checkpoints\model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.5304 - loss: 1.1342 - val_accuracy: 0.3906 - val_loss: 1.3945
Epoch 2/2
 1/19 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.3750 - loss: 1.3991

c:\Users\amanbatra\Chicken-Disease-Classification\venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_loss improved from 1.39454 to 1.33003, saving model to artifacts\prepare_callbacks\checkpoints\model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 509ms/step - accuracy: 0.3750 - loss: 1.3991 - val_accuracy: 0.6094 - val_loss: 1.3300
